# LangChain 기능 소개

- 1. 프롬프트 템플릿과 로더를 사용하여 체인 구성  
- 2. Runnable과 LangChain 표현 언어  
- 3. RAG 체인 구축  
- 4. 체인이 달린 도구 사용  

## LangChain 개요

LangChain으로 개발된 애플리케이션은 일반적으로 세 가지 범주로 나뉩니다.

- 챗봇 : LLM을 이용하여 보다 지능적인 마케팅, 고객 지원, 교육 상호작용 구현.
- 검색 증강 생성(RAG) Q&A : 대용량 문서 요약, 데이터 분석, 외부 소스를 참조하여 코드 생성에 사용.
- 에이전트 시스템은 다중 에이전트 설정과 인간 상호작용을 포함하며 복잡한 워크플로우를 위해 LangGraph를 활용. 공급망 관리 및 운영 최적화와 같은 분야에 적용 가능.

LangChain은 자연어를 실행 가능한 프로그램으로 변환하는 파이프라인을 생성할 수 있게 합니다. 이러한 체인을 활용함으로써 사용자는 자연어를 입력하고 보고서, 분석 또는 컴퓨터 프로그램과 같은 출력을 받을 수 있습니다.

In [ ]:
# !pip install -U langchain-core
# !pip install -U langchain-openai
# !pip install -U langchain-community
# !pip install -U langchain-experimental
# !pip install -U langgraph

In [ ]:
# env 파일에서 API 키를 로드

### 사용 가능한 LLM 목록 조회

In [ ]:
# 사용 가능한 모델 목록을 가져옵니다.
# 각 모델의 ID 출력

## LLM 연결

OpenAI 및 Anthropic API에 연결합니다. 원하는 모델을 지정하여 ChatOpenAI 및 ChatAnthropic 클래스의 인스턴스를 만듭니다. llm_claude3 인스턴스를 사용하여 간단한 쿼리를 호출하여 설정을 확인합니다.

In [ ]:
# from langchain_anthropic import ChatAnthropic
# llm = ChatAnthropic(model="claude-3-5-haiku-20241022")

### Messages
- AIMessage: AI(인공지능) 모델이 생성한 메시지를 나타냅니다.  
예를 들어, AI 모델이 사용자의 질문에 대해 응답을 생성할 때 이 클래스가 사용됩니다.

- HumanMessage: 사람(사용자)이 생성한 메시지를 나타냅니다.  
사용자가 입력한 텍스트나 질문 등이 여기에 해당됩니다.

- StemMessage: AI 행동을 프라이밍하기 위한 메시지.
시스템 메시지는 일반적으로 일련의 입력 메시지 중 첫 번째로 전달됩니다. 일반적으로 대화의 흐름을 제어하거나 특정 지침을 제공하기 위해 사용됩니다.

In [ ]:
# system과 human/user 메시지를 이용한 기본 요청

### 수학 선생님

##  1. 프롬프트 템플릿과 로더를 사용하여 체인 구성  

- LLM에서의 Chain 은 Data Processing 에서의 Pipeline 과 유사한 개념

### 체인의 구성 요소 - Runnables
- 프롬프트 : LLM의 응답을 안내하는 템플릿  
- LLM 또는 채팅 모델 : 프롬프트에 따라 응답을 생성하는 엔진  
- 출력 파서 : LLM의 출력을 파싱하는 도구  
- 도구 : LLM이 API에서 추가 정보를 추출하거나 코드를 실행하여 LLM을 에이전트로 전환할 수 있게 해주는 확장 기능  
- 일반 함수 : 서로 연결될 수 있는 추가적인 일반 함수

In [ ]:
# simple prompt template 생성
# {topic} 변수에서 사용자의 query 가 대체된다.
# prompt template 을 이용하여 prompt 생성 - old style
# prompt template 을 이용하여 prompt 생성 - new style

In [ ]:
# pipe operator "|"" 를 이용하여 하나 이상의 chain 을 결합

--------------------

## 2. Runnable과 LangChain 표현 언어  

- LCEL(LangChain Expression Language) 는 Runnables 를 chain 으로 구성하는 방법

LCEL은 기본 구성 요소에서 복잡한 체인을 구축하는 것을 간소화합니다. 파이프 연산자(|)를 사용하여 다양한 구성 요소를 체인으로 연결하고 한 요소에서 다음 요소로 출력을 공급합니다. 이런 방식으로 구성된 체인의 간단한 예로는 모델과 출력 파서가 결합된 프롬프트가 있습니다.  이러한 구성 요소들을 runnables 라고 부릅니다.  

`chain=prompt | model | output_parser`

- Chain 구성  
    - 체인에 대한 입력(일반적으로 사전)
    - 입력은 프롬프트로 전송됩니다.
    - 프롬프트 값은 LLM 또는 채팅 모델로 전송됩니다.
    - Chatmodel이 채팅 메시지를 반환합니다.
    - 파서는 채팅 메시지에서 문자열을 추출합니다.
    - 문자열은 체인의 출력입니다

- LangChain의 runnable 객체들:

    - RunnableSequence : 여러 runnable 구성 요소를 연결하여 각 구성 요소가 입력을 처리하고 출력을 다음 구성 요소에 전달
    - RunnableLambda : Python의 호출 가능한 요소(함수 등)를 실행 가능한 구성 요소로 바꿔서 체인으로 통합
    - RunnablePassthrough : 입력을 변경하지 않고 통과시키거나 출력에 추가 키를 추가. placeholder 역할을 하거나 시퀀스에 유연하게 통합할 수 있다.
    - RunnableParallel : 여러 개의 실행 파일을 동시에 실행하여 두 개의 체인이 동일한 입력에서 실행되지만 다른 출력을 반환하는 분기를 허용.

### Runnables

###  "|" 연산자를 이용하여 Runnable Sequence chain 생성

### RunnableLambda 사용

In [ ]:
# RunnableLambda를 사용하여 Python 함수를 체인에 삽입합니다.
# 사용자 정의 람다 함수를 정의하고 이를 RunnableLambda에 래핑합니다.

## 3. 체인이 달린 도구 사용  

- 도구는 에이전트, 체인 또는 대형 언어 모델(LLM)이 세상과 상호 작용할 수 있게 하는 인터페이스입니다.
    - Python REPL, Wikipdedia, YouTube, Zapier, Gradio, etc

In [ ]:
# !pip install --upgrade -q youtube_search

In [ ]:
# 유튜브를 검색하는 함수
# YouTubeSearchTool 인스턴스 생성
# 아무 키워드로 유튜브 검색 실행 --> 함수가 잘 실행되는 것 확인.

- YouTubeSearchTool 함수를 LLM 에 넘겨 주고 LLM이 이 함수를 이용하여 세상과 상호 작용 (유튜브 검색)할 수 있도록 함

In [ ]:
# 1단계: LLM이 어떤 tool과 args를 쓸지 판단할 수 있도록 함
# YouTube 도구를 LLM에 바인딩 --> OpenAI API 의 'tools=' parameter에 함수명과 
# 함수 호출에 필요한 parameter 정보 전달

In [ ]:
# msg에서 OpenAI LLM이 구성해준 함수호출 parameter 발췌

In [ ]:
# YouTubeSearchTool 호출에 필요한 parameter

In [ ]:
#2단계: Tool call → 파라미터 추출 → Tool 실행
# llm_with_tools에서 추출된 인수로 YouTubeSearchTool을 사용하여 유튜브 검색 실행하는 chain 생성

### Chains 종류

**LangChain**을 사용하여 다양한 체인(Chain) 및 LLM(대규모 언어 모델) 기반 애플리케이션을 구축합니다.

**1. Simple Chain (단일 체인)**  
- 하나의 프롬프트를 통해 LLM(OpenAI)을 사용하여 텍스트를 생성합니다.  


**2. Simple Sequential Chain (연속 체인)**  
-  여러 LLM 호출을 연속적으로 수행하여 출력을 다음 입력으로 전달합니다.  

**3. Document 요약 체인**  
- **목적:** 텍스트 문서를 요약합니다.  

**4. 텍스트를 Vector Store로 변환**  
- **4.1 VectortstoreIndexCreator**   
- **4.2 Chroma DB 사용**  

**5. HTTP Request Chain (웹 요청 체인)**  
- HTTP 요청을 통해 외부 웹 데이터에서 정보를 추출합니다.

### 1. Simple Chain (단일 체인)
- 가장 기본적인 유형의 체인
- 입력 프롬프트를 수신하고 이를 사용하여 텍스트를 생성하는 역할을 담당하는 언어 모델(LLM) 하나만 있습니다.

In [ ]:
# chain = prompt.pipe(llm)
# 입력 변수만 지정하여 체인을 실행

---------------------------

### 2. Simple Sequential Chains (연속 체인)
- Sequential Chain은 언어 모델에 대한 일련의 연속 호출 포함
- 이 접근 방식은 한 호출에서 생성된 출력을 다른 호출의 입력으로 활용할 때 특히 유용